# One-vs-all Classification

### J cost function

$$J(\theta) = \frac{1}{m}\sum_{i=1}^m [-y^{(i)} \log (h_{\theta}(x^{(i)}))-(1-y^{(i)})\log(1-h_{\theta}(x^{(i)}))]+ \frac{\lambda}{2m}\sum_{j=1}^{n}\theta_j^2 $$

* * *
### The gradient of the cost
\begin{align}
\frac{\partial}{\partial \theta_j}J(\theta) & = \frac{1}{m}\sum_{i=1}^m (h_{\theta}(x^{(i)})-y^{(i)})\cdot x_j^{(i)} \text{ for } j=0 \\
\frac{\partial}{\partial \theta_j}J(\theta) & = (\frac{1}{m}\sum_{i=1}^m (h_{\theta}(x^{(i)})-y^{(i)})\cdot x_j^{(i)})+ \frac{\lambda}{m}\theta_j \text{ for } j \geq 1
\end{align}


In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### If load data from a mat file
```python
import scipy.io as sio
mat_contents = sio.loadmat('digist.mat')
number_label = 10
initial_X = mat_contents['X']
print('X.shape=', initial_X.shape)
initial_y = mat_contents['y']
initial_y = np.mod(initial_y, 10)
print('y.shape=', initial_y.shape)
```

### Load digist data from scipy

In [2]:
from sklearn.datasets import load_digits

digists_dataset = load_digits()
print(digists_dataset.DESCR)
number_label = 10
initial_X = digists_dataset.data
initial_y = digists_dataset.target
print('X.shape=', initial_X.shape)
print('y.shape=', initial_y.shape)

Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 where each element is a

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(initial_X, initial_y, train_size=0.7, random_state=0)

E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
y_train.shape

(1257,)

In [5]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
def sigmoid(z):
    return 1/(1+np.exp(-1*z))

def compute_cost(X_values, y_values, theta, lambda_value):
    m = len(X_values)
    h = sigmoid(X_values.dot(theta))
    
    y1 = y_values[y_values==1]
    y0 = y_values[y_values==0]
    h1 = h[y_values==1]
    h0 = h[y_values==0]
        
    cost1 = -1*y1.T.dot(np.log(h1))
    cost0 = -((1-y0).T).dot(np.log(1-h0))    
    cost = (cost1+cost0)/m
    newtheta = np.copy(theta)
    newtheta[0]=0
    cost = cost +  lambda_value* (newtheta.T.dot(newtheta))/(2*m)
    grad = X_values.T.dot(h-y_values)/m
    grad = grad + lambda_value* newtheta/m
    return cost, grad

def compute_cost_backup(X_values, y_values, theta, lambda_value):
    m = len(X_values)
    h = sigmoid(X_values.dot(theta))
    cost = (-1*y_values.T.dot(np.log(h))-((1-y_values).T).dot(np.log(1-h)))/m
    newtheta = np.copy(theta)
    newtheta[0] = 0
    cost = cost +  lambda_value* (newtheta.T.dot(newtheta))/(2*m)
    grad = X_values.T.dot(h-y_values)/m
    grad = grad + lambda_value* newtheta/m
    return cost, grad

In [7]:
import scipy.optimize as opt

In [8]:
def costFunction(theta, X_values, y_values, lambda_value):
    theta = theta.reshape(-1, 1)
    cost, grad = compute_cost(X_values, y_values, theta, lambda_value)
    return cost

def gradFunction(theta, X_values, y_values, lambda_value):
    theta = theta.reshape(-1, 1)
    cost, grad = compute_cost(X_values, y_values, theta, lambda_value)
    grad = grad.reshape(-1)
    return grad

### Use one-vs-all method to classfiy multiclass data

In [9]:
def oneVsAll(X_values, y_values, num_labels, lambda_value):
    m = X_values.shape[0]
    n = X_values.shape[1]
    all_theta = np.zeros([num_labels, n + 1])
    X = np.insert(X_values, 0, 1, axis=1)
    
    for i in range(num_labels):
        y = (y_values==i).astype(int)
        initial_theta = np.zeros([n + 1, 1])
        print('{}-th class'.format(i))
        Result = opt.minimize(fun = costFunction, x0 = initial_theta, args = (X, y, lambda_value), 
                              method = 'BFGS', jac = gradFunction, options={'maxiter': 400, 'disp': True})
        all_theta[i] = Result.x.T.reshape(-1)
        
    return all_theta

In [10]:
def predictOneVsAll(theta, X_values):
    m = X_values.shape[0]
    num_labels = theta.shape[0]

    p = np.zeros([m,1]) # class, m X num_labels
    X = np.insert(X_values, 0, 1, axis=1)
    
    z = X.dot(theta.T) # m X num_labels = m X n * n X num_labels 
    h = sigmoid(z);
    
    pc = np.argmax(h, axis=1)
    p = pc.reshape(-1,1)
    return p

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_train)

In [12]:
thetas = oneVsAll(X_scaled, y_train.reshape(-1,1), number_label, 1)

0-th class
Optimization terminated successfully.
         Current function value: 0.029570
         Iterations: 114
         Function evaluations: 115
         Gradient evaluations: 115
1-th class
Optimization terminated successfully.
         Current function value: 0.073826
         Iterations: 125
         Function evaluations: 127
         Gradient evaluations: 127
2-th class
Optimization terminated successfully.
         Current function value: 0.039677
         Iterations: 129
         Function evaluations: 130
         Gradient evaluations: 130
3-th class
Optimization terminated successfully.
         Current function value: 0.063572
         Iterations: 139
         Function evaluations: 140
         Gradient evaluations: 140
4-th class
Optimization terminated successfully.
         Current function value: 0.040887
         Iterations: 100
         Function evaluations: 101
         Gradient evaluations: 101
5-th class
Optimization terminated successfully.
         Current func

In [13]:
X_test_scaled = scaler.transform(X_test)

In [14]:
predict = predictOneVsAll(thetas, X_test_scaled)

In [15]:
np.mean(predict==y_test.reshape(-1,1))

0.95740740740740737

> If we didn't normalize, the result is 0.94629

## sklearn.linear_model.LogisticRegression() support multiclass (by OneVsRest)

In [16]:
from sklearn import linear_model
logi_reg = linear_model.LogisticRegression(C=1)
logi_reg.fit(X_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
predict = logi_reg.predict(X_test)
np.mean(predict==y_test)

0.95370370370370372

## from sklearn.multiclass import OneVsRestClassifier

In [18]:
from sklearn.multiclass import OneVsRestClassifier

In [19]:
clf = OneVsRestClassifier(linear_model.LogisticRegression())
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [20]:
predict = clf.predict(X_test)
np.mean(predict==y_test)

0.95185185185185184